In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 38.2 MB/s eta 0:00:00


In [6]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import re

# 데이터프레임 읽기
csv_file_path = "/content/drive/MyDrive/output/crawl_merged_data.csv"
df = pd.read_csv(csv_file_path)

# null 값을 빈 문자열로 대체하기
df['Contents'].fillna('', inplace=True)

# KoNLPy의 Okt 형태소 분석기를 사용하여 명사만 추출하여 토큰화하는 함수
def model_tokenize_nouns(text):
    okt = Okt()
    text = re.sub(r"[^가-힣a-zA-Z0-9]", " ", text)
    nouns = okt.nouns(text)
    return nouns

# 데이터프레임의 텍스트 전처리 및 토큰화
df['Tokenized_Nouns'] = df['Contents'].apply(model_tokenize_nouns)

# 그룹화된 기업별로 텍스트를 모아서 Word2Vec 모델 학습
company_embeddings = {}
for company, group in df.groupby('Company'):
    merged_text = ' '.join(group['Tokenized_Nouns'].sum())
    tokens = merged_text.split()
    model = Word2Vec([tokens], vector_size=100, window=5, min_count=1, workers=4)
    company_embeddings[company] = model

# 새로운 자기소개서가 저장된 CSV 파일 경로
new_resume_csv_path = "/content/drive/MyDrive/output/new_resume.csv"

# CSV 파일 읽기
new_resume_df = pd.read_csv(new_resume_csv_path)

# 새로운 자기소개서 텍스트 추출
new_resume_text = new_resume_df['Contents'].iloc[0]

# 새로운 자기소개서 토큰화 함수
new_resume_tokens = model_tokenize_nouns(new_resume_text)
new_resume_model = Word2Vec([new_resume_tokens], vector_size=100, window=5, min_count=1, workers=4)

# 기업별 자소서와의 유사도 계산
similarities = {}
for company, model in company_embeddings.items():
    similarity = cosine_similarity([new_resume_model.wv[word] for word in new_resume_tokens if word in new_resume_model.wv],
                                   [model.wv[word] for word in new_resume_tokens if word in model.wv])
    similarities[company] = similarity[0][0]

# 유사도가 높은 순으로 정렬
sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)


# 코사인 유사도를 확률로 변환하는 softmax 함수
def softmax(similarities):
    exp_sim = np.exp(similarities)
    return exp_sim / exp_sim.sum()

# 유사도를 계산한 후 softmax 적용하여 유사도 확률 벡터 생성
similarities = np.array([similarity for _, similarity in sorted_similarities])
similarity_probabilities = softmax(similarities)
# 기업 순서 출력
company_order = [company for company, _ in sorted_similarities]
print(f"기업 순서: {company_order}")
print(f"유사도 확률 벡터: {similarity_probabilities}")
print(f"확률의 합: {sum(similarity_probabilities)}")

기업 순서: ['카카오', '우아한 형제들', '쿠팡', '라인', '네이버']
유사도 확률 벡터: [0.20594607 0.20584546 0.20079741 0.19603445 0.19137664]
확률의 합: 1.0000000298023224
